# Feature Extractions 
## Data Preparation

In [1]:
import feature
import feature_extraction
import dimension_reduction as dimension_reduction
import numpy as np

In [2]:
import pandas as pd
cal_df = pd.read_csv('calendar.csv')
price_df = pd.read_csv('sell_prices.csv')
cal_df_groups = cal_df.groupby(['event_name_1','event_type_1','event_name_2','event_type_2'])
event_info = list(cal_df_groups.groups)
df = pd.read_csv('sales_train_validation.csv')
from sklearn.preprocessing import OrdinalEncoder

def prep_calendar(df):
    df = df.drop(["weekday", "event_type_1", "event_type_2"], axis=1)
    df = df.assign(d = df.d.str[2:].astype(int))
    to_ordinal = ["event_name_1", "event_name_2"]
    df[to_ordinal] = df[to_ordinal].fillna("1")
    df[to_ordinal] = OrdinalEncoder(dtype="int").fit_transform(df[to_ordinal]) + 1
    to_int8 = ["wday", "month", "snap_CA", "snap_TX", "snap_WI"] + to_ordinal
    df[to_int8] = df[to_int8].astype("int8")

    return df

calendar = prep_calendar(cal_df)
df = df.loc[:500].copy(deep=True)
calendar_filter = calendar[['wm_yr_wk','d','event_name_1','event_name_2']]
calendar_price_df = pd.merge(calendar_filter,price_df,on='wm_yr_wk')
calendar_price_df['item_store_id'] = calendar_price_df['item_id'] +  calendar_price_df['store_id']
df['item_store_id'] = df['item_id'] + df['store_id']
selected_item_store_ls = list(pd.DataFrame(df.groupby(['item_store_id']).count()).index)
selected_calendar_price_df = calendar_price_df[calendar_price_df['item_store_id'].isin(selected_item_store_ls)]
clean_df = pd.merge(df,selected_calendar_price_df.groupby(['item_store_id']).agg('mean')['sell_price'],on='item_store_id')

In [3]:
cal_df

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1964,2016-06-15,11620,Wednesday,5,6,2016,d_1965,NaN,NaN,NaN,NaN,0,1,1
1965,2016-06-16,11620,Thursday,6,6,2016,d_1966,NaN,NaN,NaN,NaN,0,0,0
1966,2016-06-17,11620,Friday,7,6,2016,d_1967,NaN,NaN,NaN,NaN,0,0,0
1967,2016-06-18,11621,Saturday,1,6,2016,d_1968,NaN,NaN,NaN,NaN,0,0,0


In [4]:
price_df

,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.58
1,CA_1,HOBBIES_1_001,11326,9.58
2,CA_1,HOBBIES_1_001,11327,8.26
3,CA_1,HOBBIES_1_001,11328,8.26
4,CA_1,HOBBIES_1_001,11329,8.26
...,...,...,...,...
6841116,WI_3,FOODS_3_827,11617,1.00
6841117,WI_3,FOODS_3_827,11618,1.00
6841118,WI_3,FOODS_3_827,11619,1.00
6841119,WI_3,FOODS_3_827,11620,1.00


In [5]:
df

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913,item_store_id
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,3,0,1,1,1,3,0,1,1,HOBBIES_1_001CA_1
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,1,0,0,0,0,HOBBIES_1_002CA_1
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,2,1,1,1,0,1,1,1,HOBBIES_1_003CA_1
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,5,4,1,0,1,3,7,2,HOBBIES_1_004CA_1
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,1,0,1,1,2,2,2,4,HOBBIES_1_005CA_1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
496,HOBBIES_2_081_CA_1_validation,HOBBIES_2_081,HOBBIES_2,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,0,1,0,0,HOBBIES_2_081CA_1
497,HOBBIES_2_082_CA_1_validation,HOBBIES_2_082,HOBBIES_2,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,0,HOBBIES_2_082CA_1
498,HOBBIES_2_083_CA_1_validation,HOBBIES_2_083,HOBBIES_2,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,1,0,0,0,0,0,0,HOBBIES_2_083CA_1
499,HOBBIES_2_084_CA_1_validation,HOBBIES_2_084,HOBBIES_2,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,0,HOBBIES_2_084CA_1


In [6]:
clean_df

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913,item_store_id,sell_price
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,1,1,1,3,0,1,1,HOBBIES_1_001CA_1,8.285275
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,1,0,0,0,0,HOBBIES_1_002CA_1,3.970000
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,1,1,0,1,1,1,HOBBIES_1_003CA_1,2.970000
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,5,4,1,0,1,3,7,2,HOBBIES_1_004CA_1,4.528159
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,1,1,2,2,2,4,HOBBIES_1_005CA_1,2.941066
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
496,HOBBIES_2_081_CA_1_validation,HOBBIES_2_081,HOBBIES_2,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,1,0,0,HOBBIES_2_081CA_1,2.057199
497,HOBBIES_2_082_CA_1_validation,HOBBIES_2_082,HOBBIES_2,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,HOBBIES_2_082CA_1,0.880000
498,HOBBIES_2_083_CA_1_validation,HOBBIES_2_083,HOBBIES_2,HOBBIES,CA_1,CA,0,0,0,0,...,0,1,0,0,0,0,0,0,HOBBIES_2_083CA_1,2.771083
499,HOBBIES_2_084_CA_1_validation,HOBBIES_2_084,HOBBIES_2,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,HOBBIES_2_084CA_1,6.312553


In [7]:
clean_df.to_json(r'cleandf_json.json')

## Split attributes and timeseries 

In [8]:
attributes = ['id','item_id','dept_id','cat_id','store_id','state_id','item_store_id','sell_price']
att_df = clean_df[attributes]
tse_df = clean_df.loc[:, ~clean_df.columns.isin(attributes)]

In [9]:
att_df.head(3)

,id,item_id,dept_id,cat_id,store_id,state_id,item_store_id,sell_price
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,HOBBIES_1_001CA_1,8.285275
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,HOBBIES_1_002CA_1,3.970000
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,HOBBIES_1_003CA_1,2.970000


In [10]:
tse_df.head(3)

,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,0,0,0,0,0,0,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1


## EDA Attributes Generation 

### Group attributes by multi-level groups 

In [11]:
# Group certain level, apply EDA for numerical features in each groups.
# In our case, numerical features will be average price. 
import itertools 
groupby_attributes = ['state_id','store_id','cat_id','dept_id','item_id']
all_combinations = []
for r in range(len(groupby_attributes) + 1):
    if r==0:
        continue
    combinations_object = itertools.combinations(groupby_attributes, r)
    combinations_list = list(combinations_object)
    all_combinations += combinations_list

selected_group = []
for group_level in all_combinations:
    groups = att_df.groupby(list(group_level))
    for group in groups:
        num_tse = len(group[1])
        if num_tse > 1:
#             print('Num tse:'+ str(num_tse) +'(group level:' + str(group_level) + ')')
            selected_group.append(group_level)
            

### Exploit attribute extraction function that return tuple

In [12]:
import feature_extraction.builtin as ft
import feature_extraction.freqbased as fb
from feature import * 
models = [ft.most_least_frequent]
att_df = gen_attribute_tuple_based(models, selected_group, att_df, col='sell_price')

### Exploit attribute extraction function that return table

In [13]:
top_k = 3
models = [ft.count_c, ft.count_r]
att_df = gen_attribute_table_based(models, selected_group, att_df, col='sell_price')

### Exploit attribute extraction function that return list

In [14]:
models = [ft.outlier, ft.uniq_values, fb.outliers_freq]
att_df = gen_attribute_list_based(models, selected_group, att_df,col='sell_price')

### Exploit attribute extraction function that return value

In [15]:
models = [ft.min, ft.max, ft.outlier_c, ft.zero_c, 
         ft.null_c, ft.null_r, ft.mean, ft.median, 
         ft.cardinality_r, ft.cardinality_c, ft.variance, 
          ft.std_dev, ft.range, ft.coef_of_var,
         ft.sum, ft.kurtosis, ft.skewness,
          fb.min_freq,fb.max_freq,fb.median_freq,fb.variance_freq,
         fb.stdev_freq, fb.coef_of_var_freq, 
          fb.range_freq, fb.outlier_c_freq, 
          ft.iqr,fb.mean_abs_dev_freq,
         ft.mean_abs_dev]
att_df = gen_attribute_value_based(models, selected_group, att_df, col='sell_price')

### Exploit attribute extraction function that return series

In [16]:
models = [ft.mode]
att_df = gen_attribute_series_based(models, selected_group, att_df, col='sell_price')

### Exploit quantile attribute extraction function

In [17]:
models_quantile = {'1stQuartile':0.25,
                   '2ndQuartile':0.5,
                   '3rdQuartile':0.75,
                   '4thQuartile':1,
                   '1stPercentile':0.01,
                   '10stPercentile':0.10,
                   '90stPercentile':0.90,
                   '99stPercentile':0.99
                  }
att_df = gen_attribute_quantile(models_quantile, selected_group, att_df, col='sell_price')

### Convert ID to categorical 

In [18]:
att_df = text2cat(att_df, 'id','id_cat')
att_df = text2cat(att_df, 'item_id','item_id_cat')
att_df = text2cat(att_df, 'dept_id','dept_id_cat')
att_df = text2cat(att_df, 'cat_id','cat_id_cat')
att_df = text2cat(att_df, 'store_id','store_id_cat')
att_df = text2cat(att_df, 'state_id','state_id_cat')
att_df = text2cat(att_df, 'item_store_id','item_store_id_cat')

In [19]:
att_df = att_df.drop(['id','item_id','dept_id','cat_id','store_id','state_id','item_store_id'],axis=1)

In [20]:
att_df.columns

Index(['sell_price', 'store_id_sell_price_most_least_frequent_top-0',
       'store_id_sell_price_most_least_frequent_top-1',
       'store_id_sell_price_most_least_frequent_top-2',
       'store_id_sell_price_most_least_frequent_bottom-0',
       'store_id_sell_price_most_least_frequent_bottom-1',
       'store_id_sell_price_most_least_frequent_bottom-2',
       'cat_id_dept_id_sell_price_most_least_frequent_top-0',
       'cat_id_dept_id_sell_price_most_least_frequent_top-1',
       'cat_id_dept_id_sell_price_most_least_frequent_top-2',
       ...
       'state_id_store_id_cat_id_sell_price_99stPercentile',
       'store_id_cat_id_dept_id_sell_price_99stPercentile',
       'store_id_dept_id_sell_price_99stPercentile', 'id_cat', 'item_id_cat',
       'dept_id_cat', 'cat_id_cat', 'store_id_cat', 'state_id_cat',
       'item_store_id_cat'],
      dtype='object', length=4915)

## Timeseries Features Generation

In [21]:
df = pd.DataFrame()

### Exploit function that apply for timeseries dataframe return value 

In [22]:
import feature_extraction.tse_features as tse
models = [tse.min, tse.max, tse.calc_range, tse.iqr, 
          tse.median, tse.mode, tse.mean, tse.nz_mean, tse.var,
          tse.nz_var, tse.std, tse.nz_std, tse.cv,
         tse.min_index, tse.max_index, tse.max_freq,tse.min_freq,
         tse.adi, tse.cvsq_id, tse.skewness, tse.kurtosis, 
          tse.normaltest_pvalue,tse.normaltest_stats]
df = tse_features(tse_df, df, models)

### Exploit function that apply for date series and return value

In [23]:
from datetime import datetime
date_df = cal_df.iloc[:1913,0]
date_series = date_df.apply(lambda x: datetime.strptime(x,'%Y-%m-%d'))
date_series = pd.Series(date_series, index = date_series)
models = [tse.years, tse.weeks, tse.months, tse.days, tse.count_weekends, tse.count_weekdays]
df = tse_features_datetime(date_series, df, models)

### Exploit function that apply for date series and need input parameters

In [24]:
models = [tse.data_points]
paras = [{'Days': 'D',
         'Yrs': 'Y',
         'Months': 'M',
         'Weeks': 'W'}]
df = tse_features_datetime_para(date_series,models,paras, df)

### Exploit function that apply for timeseries dataframe and need input parameters, date information

In [25]:
models = [tse.trend_value, tse.ljbox_stat_value, tse.ljbox_p_value]
trend_dict = {',MA': 'MA',
             'LOESS': 'LOESS'}
ljbox_dict =  {'lag1':1,
             'lag2':2}
ljbox_p_dict =  {'lag1':1,
             'lag2':2}

paras = [trend_dict, ljbox_dict, ljbox_p_dict]
df = tse_features_with_time_para(date_series, df, tse_df, models, paras)

/home/jupyter-amarjyot-kaur/.conda/envs/AjkvirtualEnv/lib/python3.8/site-packages/statsmodels/stats/diagnostic.py:494: FutureWarning: The default value of lags is changing.  After 0.12, this value will become min(10, nobs//5). Directly setlags to silence this warning.
  warnings.warn("The default value of lags is changing.  After 0.12, "
/home/jupyter-amarjyot-kaur/.conda/envs/AjkvirtualEnv/lib/python3.8/site-packages/statsmodels/stats/diagnostic.py:524: FutureWarning: The value returned will change to a single DataFrame after 0.12 is released.  Set return_df to True to use to return a DataFrame now.  Set return_df to False to silence this warning.
  warnings.warn(msg, FutureWarning)


### Exploit function that apply for timeseries dataframe and need input parameters, date information

In [26]:
box_dict =  {'lag1':1,
             'lag2':2}
box_p_dict =  {'lag1':1,
             'lag2':2}
paras = [box_dict, box_p_dict]
models = [tse.boxpierce_stat_value, tse.boxpierce_p_value]
df = tse_features_with_time_para_box(date_series, df, tse_df, models, paras)

### Exploit function that apply for timeseries dataframe and need input parameters

In [27]:
quantile_dict = {'1stQuartile':0.25,
                   '2ndQuartile':0.5,
                   '3rdQuartile':0.75,
                   '4thQuartile':1,
                   '1stPercentile':0.01,
                   '10stPercentile':0.10,
                   '90stPercentile':0.90,
                   '99stPercentile':0.99
                  }
pacf_dict = {'lag1':1,
             'lag2':2}
acovf_dict = {'lag1':1,
             'lag2':2}

paras = [quantile_dict,acovf_dict, pacf_dict]
models = [tse.quantile, tse.acovf, tse.pacf]
df = tse_features_para(tse_df,df, models, paras)

/home/jupyter-amarjyot-kaur/.conda/envs/AjkvirtualEnv/lib/python3.8/site-packages/statsmodels/tsa/stattools.py:372: FutureWarning: fft=True will become the default in a future version of statsmodels. To suppress this warning, explicitly set fft=False.
  warnings.warn(msg, FutureWarning)


### Exploit function that apply for timeseries dataframe and need date information

In [28]:
models = [tse.durbinwatson_test]
df = tse_features_with_time(date_series, df, tse_df, models)

In [29]:
df.columns

Index(['min', 'max', 'calc_range', 'iqr', 'median', 'mode', 'mean', 'nz_mean',
       'var', 'nz_var', 'std', 'nz_std', 'variation', 'min_index', 'max_index',
       'max_freq', 'min_freq', 'adi', 'cvsq_id', 'skewness', 'kurtosis',
       'normaltest_pvalue', 'normaltest_stats', 'years', 'weeks', 'months',
       'days', 'count_weekends', 'count_weekdays', 'data_points_Days',
       'data_points_Yrs', 'data_points_Months', 'data_points_Weeks',
       'trend_value_,MA', 'trend_value_LOESS', 'ljbox_stat_value_lag1',
       'ljbox_stat_value_lag2', 'ljbox_p_value_lag1', 'ljbox_p_value_lag2',
       'boxpierce_stat_value_lag1', 'boxpierce_stat_value_lag2',
       'boxpierce_p_value_lag1', 'boxpierce_p_value_lag2',
       'quantile_1stQuartile', 'quantile_2ndQuartile', 'quantile_3rdQuartile',
       'quantile_4thQuartile', 'quantile_1stPercentile',
       'quantile_10stPercentile', 'quantile_90stPercentile',
       'quantile_99stPercentile', 'acovf_lag1', 'acovf_lag2', 'pacf_lag1',
       '

## Feature Concatenation

In [30]:
feature = pd.concat([att_df,df],axis=1)
feature = feature.reset_index()

In [31]:
# Drop categorical information 
feature = feature.drop(['id_cat','item_id_cat','dept_id_cat',
                        'cat_id_cat','store_id_cat','state_id_cat','item_store_id_cat'],axis=1)

In [32]:
feature

,index,sell_price,store_id_sell_price_most_least_frequent_top-0,store_id_sell_price_most_least_frequent_top-1,store_id_sell_price_most_least_frequent_top-2,store_id_sell_price_most_least_frequent_bottom-0,store_id_sell_price_most_least_frequent_bottom-1,store_id_sell_price_most_least_frequent_bottom-2,cat_id_dept_id_sell_price_most_least_frequent_top-0,cat_id_dept_id_sell_price_most_least_frequent_top-1,...,quantile_4thQuartile,quantile_1stPercentile,quantile_10stPercentile,quantile_90stPercentile,quantile_99stPercentile,acovf_lag1,acovf_lag2,pacf_lag1,pacf_lag2,durbinwatson_test
0,0,8.285275,1.764774,0.476018,4.97,2.771083,6.312553,0.97,1.764774,0.476018,...,5.0,0.0,0.0,1.0,3.00,0.108222,0.099448,0.230713,0.167824,2.295230
1,1,3.970000,1.764774,0.476018,4.97,2.771083,6.312553,0.97,1.764774,0.476018,...,5.0,0.0,0.0,1.0,2.88,0.040712,0.044337,0.125693,0.123101,2.396553
2,2,2.970000,1.764774,0.476018,4.97,2.771083,6.312553,0.97,1.764774,0.476018,...,6.0,0.0,0.0,1.0,2.00,0.068358,0.069994,0.289055,0.231951,2.444184
3,3,4.528159,1.764774,0.476018,4.97,2.771083,6.312553,0.97,1.764774,0.476018,...,15.0,0.0,0.0,4.0,9.00,0.430308,-0.029662,0.108789,-0.019570,2.349615
4,4,2.941066,1.764774,0.476018,4.97,2.771083,6.312553,0.97,1.764774,0.476018,...,9.0,0.0,0.0,3.0,5.00,0.399263,0.230941,0.238522,0.086040,2.227697
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
496,496,2.057199,1.764774,0.476018,4.97,2.771083,6.312553,0.97,0.970000,0.970000,...,4.0,0.0,0.0,0.0,1.00,0.006214,0.004643,0.065029,0.044576,2.360460
497,497,0.880000,1.764774,0.476018,4.97,2.771083,6.312553,0.97,0.970000,0.970000,...,4.0,0.0,0.0,0.0,1.88,0.008532,0.002780,0.084909,0.020620,2.311088
498,498,2.771083,1.764774,0.476018,4.97,2.771083,6.312553,0.97,0.970000,0.970000,...,16.0,0.0,0.0,1.0,4.00,0.051661,0.116429,0.058546,0.129028,2.479697
499,499,6.312553,1.764774,0.476018,4.97,2.771083,6.312553,0.97,0.970000,0.970000,...,9.0,0.0,0.0,0.0,1.00,0.008600,0.003894,0.073008,0.027891,2.320634


In [33]:
feature.shape

(501, 4965)

In [34]:
feature = np.array(feature)

In [35]:
feature.shape

(501, 4965)

In [36]:
feature

array([[0.00000000e+00, 8.28527493e+00, 1.76477400e+00, ...,
        2.30713129e-01, 1.67823644e-01, 2.29522991e+00],
       [1.00000000e+00, 3.97000000e+00, 1.76477400e+00, ...,
        1.25692789e-01, 1.23100542e-01, 2.39655299e+00],
       [2.00000000e+00, 2.97000000e+00, 1.76477400e+00, ...,
        2.89054537e-01, 2.31950866e-01, 2.44418370e+00],
       ...,
       [4.98000000e+02, 2.77108316e+00, 1.76477400e+00, ...,
        5.85458396e-02, 1.29028449e-01, 2.47969743e+00],
       [4.99000000e+02, 6.31255319e+00, 1.76477400e+00, ...,
        7.30080043e-02, 2.78906459e-02, 2.32063446e+00],
       [5.00000000e+02, 9.70000000e-01, 1.76477400e+00, ...,
        1.19041087e-01, 2.75339706e-02, 2.28169743e+00]])

# Dimension Reduction

1. Goal: represent instances with fewer variables
    - Try to preserve as much structure in the data as possible 
    - Discriminative: only structure that affects class separability 
2. Type
    - Feature selection: pick a subset of the original dimensions 
        - Discriminative: pick good class 'predictors'
        - Type: LVF, HCF, RF
    - Feature extraction: construct a new set of dimensions / (linear) combinations of original features
        - Type: FA, PCA, SVD

## High Correlation Filter

Calculate the correlation between independent numerical variables that are numerical in nature. If the correlation coefficient crosses a certain threshold value, we can drop one of the variables (dropping a variable is highly subjective and should always be done keeping the domain in mind).

Reason: high correlation between two variables means they have similar trends and are likely to carry similar information. This can bring down the performance of some models drastically (linear and logistic regression models, for instance). 

In [37]:
import dimension_reduction as dimension_reduction
dr_model = dimension_reduction.DimensionReduction(model_name = 'high_correlation_filter', threshold = 0.95, n_components = None)
dr_model.fit(feature)
feature_hcf = dr_model.transform()
print(feature_hcf.shape)

(501, 501)


In [38]:
feature_hcf

,0,1,2,8,334,393,639,645,719,987,...,4944,4945,4946,4947,4952,4956,4957,4960,4962,4963
0,0.0,8.285275,1.764774,1.764774,4.528159,1.25,3.000584,14.193601,0.5,4.528159,...,43.543999,121.959184,4.145335e-11,3.287757e-27,0.0,0.0,0.0,0.108222,0.230713,0.167824
1,1.0,3.970000,1.764774,1.764774,4.528159,1.25,3.000584,14.193601,0.5,4.528159,...,75.344040,156.737794,3.954403e-18,9.221900e-35,0.0,0.0,0.0,0.040712,0.125693,0.123101
2,2.0,2.970000,1.764774,1.764774,4.528159,1.25,3.000584,14.193601,0.5,4.528159,...,95.656833,187.785508,1.366273e-22,1.670686e-41,0.0,0.0,0.0,0.068358,0.289055,0.231951
3,3.0,4.528159,1.764774,1.764774,4.528159,1.25,3.000584,14.193601,0.5,4.528159,...,58.678730,182.161098,1.856375e-14,2.781123e-40,0.0,0.0,0.0,0.430308,0.108789,-0.019570
4,4.0,2.941066,1.764774,1.764774,4.528159,1.25,3.000584,14.193601,0.5,4.528159,...,24.859125,128.915140,6.167641e-07,1.014923e-28,0.0,0.0,0.0,0.399263,0.238522,0.086040
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
496,496.0,2.057199,1.764774,0.970000,4.528159,1.25,3.000584,14.193601,0.5,4.528159,...,62.322889,162.915732,2.915171e-15,4.200474e-36,0.0,0.0,0.0,0.006214,0.065029,0.044576
497,497.0,0.880000,1.764774,0.970000,4.528159,1.25,3.000584,14.193601,0.5,4.528159,...,46.359012,170.410296,9.845365e-12,9.905454e-38,0.0,0.0,0.0,0.008532,0.084909,0.020620
498,498.0,2.771083,1.764774,0.970000,4.528159,1.25,3.000584,14.193601,0.5,4.528159,...,110.230498,149.516485,8.723410e-26,3.411205e-33,0.0,0.0,0.0,0.051661,0.058546,0.129028
499,499.0,6.312553,1.764774,0.970000,4.528159,1.25,3.000584,14.193601,0.5,4.528159,...,49.245123,160.964948,2.258943e-12,1.114052e-35,0.0,0.0,0.0,0.008600,0.073008,0.027891


## Low Variance Filter 

Calculate the variance of each variable we are given. 

Drop the variables having low variance as compared to other variables in our dataset. 

Reason: variables with a low variance will not affect the target variable.|

In [39]:
dr_model = dimension_reduction.DimensionReduction(model_name = 'low_variance_filter', threshold = None, n_components = None)
dr_model.fit(feature)
feature_lvf = dr_model.transform()
print(feature_lvf.shape)
feature_lvf

Feature Importance by Var: 
       Feature       Variance
0        4578   8.993166e+05
1        4575   8.993166e+05
2        4570   8.993166e+05
3        4569   8.993166e+05
4        4567   8.993166e+05
...       ...            ...
4486     3048   7.719127e-32
4487     3557   7.719127e-32
4488     1737   7.719127e-32
4489     1156   7.719127e-32
4490     4930  7.608772e-106

[4491 rows x 2 columns]
(501, 4491)


,0,1,2,3,4,5,6,7,8,9,...,4953,4954,4955,4958,4959,4960,4961,4962,4963,4964
0,0.0,8.285275,1.764774,0.476018,4.97,2.771083,6.312553,0.97,1.764774,0.476018,...,0.0,0.0,5.0,1.0,3.00,0.108222,0.099448,0.230713,0.167824,2.295230
1,1.0,3.970000,1.764774,0.476018,4.97,2.771083,6.312553,0.97,1.764774,0.476018,...,0.0,0.0,5.0,1.0,2.88,0.040712,0.044337,0.125693,0.123101,2.396553
2,2.0,2.970000,1.764774,0.476018,4.97,2.771083,6.312553,0.97,1.764774,0.476018,...,0.0,0.0,6.0,1.0,2.00,0.068358,0.069994,0.289055,0.231951,2.444184
3,3.0,4.528159,1.764774,0.476018,4.97,2.771083,6.312553,0.97,1.764774,0.476018,...,1.0,2.0,15.0,4.0,9.00,0.430308,-0.029662,0.108789,-0.019570,2.349615
4,4.0,2.941066,1.764774,0.476018,4.97,2.771083,6.312553,0.97,1.764774,0.476018,...,1.0,2.0,9.0,3.0,5.00,0.399263,0.230941,0.238522,0.086040,2.227697
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
496,496.0,2.057199,1.764774,0.476018,4.97,2.771083,6.312553,0.97,0.970000,0.970000,...,0.0,0.0,4.0,0.0,1.00,0.006214,0.004643,0.065029,0.044576,2.360460
497,497.0,0.880000,1.764774,0.476018,4.97,2.771083,6.312553,0.97,0.970000,0.970000,...,0.0,0.0,4.0,0.0,1.88,0.008532,0.002780,0.084909,0.020620,2.311088
498,498.0,2.771083,1.764774,0.476018,4.97,2.771083,6.312553,0.97,0.970000,0.970000,...,0.0,0.0,16.0,1.0,4.00,0.051661,0.116429,0.058546,0.129028,2.479697
499,499.0,6.312553,1.764774,0.476018,4.97,2.771083,6.312553,0.97,0.970000,0.970000,...,0.0,0.0,9.0,0.0,1.00,0.008600,0.003894,0.073008,0.027891,2.320634


## Factor Analysis

- n_components will decide the number of factors in the transformed data

Reason: In the Factor Analysis technique, variables are grouped by their correlations, 
i.e., all variables in a particular group will have a high correlation among themselves, 
but a low correlation with variables of other group(s). Here, each group is known as a factor.
These factors are small in number as compared to the original dimensions of the data.
However, these factors are difficult to observe.

In [40]:
dr_model = dimension_reduction.DimensionReduction(model_name = 'factor_analysis', threshold = None, n_components = 3)
dr_model.fit(feature)
feature_fa = dr_model.transform()
print(feature_fa.shape)
feature_fa

Loading Scores: 
       Feature            f1            f2            f3
0           0  9.402123e+01 -3.012326e+00  5.816282e+00
1           1 -1.438191e+00 -5.179642e-01 -1.643044e+00
2           2  1.873740e-28  2.026987e-29 -1.075762e-24
3           3  2.009838e-28 -3.264746e-29  1.812065e-26
4           4 -1.801130e-28 -1.457272e-29 -1.213773e-25
...       ...           ...           ...           ...
4960     4960 -3.999806e-01  7.030661e-02  3.005697e+00
4961     4961 -3.451834e-01 -3.156428e-03  2.606976e+00
4962     4962 -1.645315e-02 -3.340558e-02  1.484958e-02
4963     4963 -8.544803e-03 -2.462425e-02  8.853067e-03
4964     4964  1.939899e-03  2.725884e-03 -5.944249e-03

[4965 rows x 4 columns]
Explained Variance Ratio: [0.9439375814458518, 0.04800240109045239, 0.008060017463695703]
Exaplined Variance Ratio with NOISE: [0.9125439308849674, 0.04640592836223575, 0.00779195591307674] 

Importance Scores:  
       Feature       Score
0        4575  864.522339
1        4578  864.

array([[-0.45202536, -0.87918119, -0.37963854],
       [-0.45202536, -1.47556138, -0.36973593],
       [-0.45202536, -1.60535106, -0.40403034],
       ...,
       [ 2.21226528,  1.07771394,  0.03010573],
       [ 2.21226528, -0.00517036, -0.11711796],
       [ 2.21226528, -0.8523408 ,  0.22366695]])

## Principal Component Analysis

- n_components will decide the number of principal components in the transformed data

Reason: PCA is a technique which helps us in extracting a new set of variables from an existing large set of variables. These newly extracted variables are called Principal Components. 

- A principal component is a linear combination of the original variables
- Principal components are extracted in such a way that the first principal component explains maximum variance in the dataset
- Second principal component tries to explain the remaining variance in the dataset and is uncorrelated to the first principal component
- Third principal component tries to explain the variance which is not explained by the first two principal components and so on

In [41]:
dr_model = dimension_reduction.DimensionReduction(model_name = 'principal_component_analysis', threshold = None, n_components = 3)
dr_model.fit(feature)
feature_pca = dr_model.transform()

feature_pca

explained variance ratio:  [0.90305051 0.05330345 0.03296517] 

cumulative explained var: [0.90305051 0.95635397 0.98931914] 

      Feature           PC1           PC2           PC3
0           0  3.458209e-02  1.322081e-03  8.391192e-03
1           1  5.327570e-04  5.875662e-05  1.710018e-03
2           2  1.354602e-31  3.940979e-22  1.483148e-21
3           3  1.133117e-32  4.551231e-28  5.676995e-27
4           4  1.963566e-33  6.126794e-31  1.790074e-31
...       ...           ...           ...           ...
4960     4960  1.518512e-04  5.384970e-04  6.069537e-04
4961     4961  1.307917e-04  3.908798e-04  6.050772e-04
4962     4962  6.202486e-06  1.158088e-05  8.714068e-05
4963     4963  3.167336e-06  1.686042e-05  5.037444e-05
4964     4964  8.009938e-07  6.323961e-06  1.897939e-05

[4965 rows x 4 columns]
Importance Scores:  
       Feature     Score
0        4567  0.316954
1        4569  0.316954
2        4566  0.316954
3        4565  0.316954
4        4570  0.316954
...       

array([[-1218.24080757,   419.09544196,  -334.77532594],
       [-1193.16695707,   761.65552691,  -490.11815649],
       [-1117.61562838,  1222.43936022,  -122.49006417],
       ...,
       [ 6032.26214719,  -349.27549484,   596.87171055],
       [ 6212.78442348,  1012.73281414,  1091.54173632],
       [ 6002.02576072,   150.79014693,  -551.27347812]])

# Feature Scaling 

In [42]:
from sklearn.preprocessing import StandardScaler
def standard_scaling(feature):
    data = np.array(feature)
    scaler = StandardScaler()
    scaler.fit(data)
    return scaler.transform(data)

In [43]:
feature = standard_scaling(feature)
feature_pca = standard_scaling(feature_pca)
feature_fa = standard_scaling(feature_fa)
feature_lvf = standard_scaling(feature_lvf)
feature_hcf = standard_scaling(feature_hcf)

# Clustering Validity 

Hopkins statistics. 
    Test the hypothesis of the existence of patterns in the data versus a dataset uniformly distributed.

Approaches:
    - Sample points from dataset uniformly and compute distance to their neareset neighbors.
    - Generate points uniformly distributed in space of dataset and compute distance to nearest neighbors.
    - Compute quoitent H.

In [ ]:
from cluster_validity import * 

In [ ]:
print('raw features:' + str(hopkins_stats(feature)))
print('pca features:' + str(hopkins_stats(feature_pca)))
print('fa features:' + str(hopkins_stats(feature_fa)))
print('lvf features:' + str(hopkins_stats(feature_lvf)))
print('hcf features:' + str(hopkins_stats(feature_hcf)))

# Confirm Number of Clusters 

In [ ]:
class Cluster():
    def __init__(self, feature, k_clusters):
        self.feature = feature
        self.k_clusters = k_clusters
        self.out_kmeans = self.clustering('KMEANS')
        self.out_kmedoids = self.clustering('KMEDOIDS')
        self.out_agglo = self.clustering('AGGLO')        
        
    def clustering(self, algo_type):
        if algo_type == 'KMEANS':
            import cluster_sklearn_kmeans as cluster
            cluster_model = cluster.SKKMeans(n_clusters=self.k_clusters)
            cluster_model.fit(self.feature)
        elif algo_type == 'KMEDOIDS':
            import cluster_sklearn_kmedoids as cluster
            cluster_model = cluster.SKKMedoids(n_clusters=self.k_clusters)
            cluster_model.fit(self.feature)        
        elif algo_type == 'AGGLO':
            import cluster_sklearn_agglomerative as cluster
            cluster_model = cluster.SKAgglomerative(n_clusters=self.k_clusters)
            cluster_model.fit(self.feature)                
        return cluster_model.transform()

In [ ]:
k = [3,5,7,9]

In [ ]:
import warnings
warnings.filterwarnings("ignore")
df_ls = []
name_list = ['feature_kmeans','feature_kmedoids','feature_agg',
                      'pca_kmeans','pca_kmedoids','pca_agg',
                      'fa_kmeans', 'fa_kmedoids', 'fa_agg',
                      'lvf_kmeans', 'lvf_kmedoids', 'lvf_agg',
                      'hcf_kmeans', 'hcf_kmedoids', 'hcf_agg']
feature_list = [feature, feature, feature,
               feature_pca, feature_pca, feature_pca,
               feature_fa, feature_fa, feature_fa,
               feature_lvf, feature_lvf, feature_lvf,
               feature_hcf, feature_hcf, feature_hcf]
for k_clusters in k:
    df = pd.DataFrame(columns= name_list)
    m = Cluster(feature, k_clusters)
    df['feature_kmeans'], df['feature_kmedoids'], df['feature_agg'] = m.out_kmeans, m.out_kmedoids, m.out_agglo    
    m = Cluster(feature_pca, k_clusters)
    df['pca_kmeans'], df['pca_kmedoids'], df['pca_agg'] = m.out_kmeans, m.out_kmedoids, m.out_agglo
    m = Cluster(feature_fa, k_clusters)
    df['fa_kmeans'], df['fa_kmedoids'], df['fa_agg'] = m.out_kmeans, m.out_kmedoids, m.out_agglo
    m = Cluster(feature_lvf, k_clusters)
    df['lvf_kmeans'], df['lvf_kmedoids'], df['lvf_agg'] = m.out_kmeans, m.out_kmedoids, m.out_agglo
    m = Cluster(feature_hcf, k_clusters)
    df['hcf_kmeans'], df['hcf_kmedoids'], df['hcf_agg'] = m.out_kmeans, m.out_kmedoids, m.out_agglo
    df_ls.append(df)

Calculate average sse for different number of k

In [ ]:
results = pd.DataFrame(columns=['Feature','Cluster','Name','K','Score'])
c = 0
for idx0, df in enumerate(df_ls):
    k_cluster = k[idx0] 
    for idx, name in enumerate(name_list):
        info = pd.DataFrame(np.array(df[name]), columns = ['cluster'])
        score = computeSSE(feature_list[idx], info)
        results.loc[c] = [name.split('_')[0],name.split('_')[1],name,k_cluster,score]
        c += 1 

In [ ]:
results.groupby(['K']).agg('mean')

We select 9 clusters.

# Cluster Evaluation

- External Index: Used to measure the extent to which cluster labels match externally supplied class labels.
    * Entropy
- Internal Index: Used to measure the goodness of a clustering structure without respect to external information.
    * Sum of Squared Error (SSE)
- Relative Index: Used to compare two different clusterings or clusters.
    * Often an external or internal index is used for this function, e.g., SSE or entropy
    
**In this library: we mainly focus on internal index and relative index.**

In [ ]:
df = df_ls[-1]

In [ ]:
results = pd.DataFrame(columns=['Feature','Cluster','Name',
                                'Sihouette_score','DaviesBouldin_score',
                                'CalinskiHarabasz_score','InterClassDistance', 'IntraClassDistance','sse'])
c = 0 
for idx, name in enumerate(name_list):
    Sihouette_score = SihouetteCoefficient(feature_list[idx], np.array(df[name]))
    DaviesBouldin_score = DaviesBouldin(feature_list[idx], np.array(df[name]))                                       
    CalinskiHarabasz_score = CalinskiHarabasz(feature_list[idx], np.array(df[name]))      
    info = pd.DataFrame(np.array(df[name]), columns = ['cluster'])                                          
    InterClassDistance = interclassDistance(feature_list[idx], info)                 
    IntraClassDistance = intraclassDistance(feature_list[idx], info)   
    sse = computeSSE(feature_list[idx],info)
    results.loc[c] = [name.split('_')[0],name.split('_')[1],name,
                      Sihouette_score,DaviesBouldin_score,CalinskiHarabasz_score,
                     InterClassDistance, IntraClassDistance, sse]
    c += 1 

In [43]:
results.head(5)

,Feature,Cluster,Name,Sihouette_score,DaviesBouldin_score,CalinskiHarabasz_score,InterClassDistance,IntraClassDistance,sse
0,feature,kmeans,feature_kmeans,0.247317,1.486733,3035.738793,521006.114283,10554.885717,1.696473e+06
1,feature,kmedoids,feature_kmedoids,0.216579,1.812765,2696.995064,519709.972268,11851.027732,1.696475e+06
2,feature,agg,feature_agg,0.256010,1.210290,3377.010997,522053.686651,9507.313349,1.696469e+06
3,pca,kmeans,pca_kmeans,0.388267,0.854907,372.052803,1289.797596,213.202404,1.095065e+03
4,pca,kmedoids,pca_kmedoids,0.386156,0.773924,303.459345,1249.726584,253.273416,1.090854e+03


## Sihouette Coefficient

- The best value is 1 and the worst value is -1. 

- Values near 0 indicate overlapping clusters.

- Negative values generally indicate that a sample has been assigned to the wrong cluster, as a different cluster is more similar.

Definition:

- a: The mean distance between a sample and all other points in the same class.
- b: The mean distance between a sample and all other points in the next nearest cluster.
- $s=\dfrac{b-a}{max(a,b)}$


Pro:
- The score is bounded between -1 for incorrect clustering and +1 for highly dense clustering. Scores around zero indicate overlapping clusters.
- The score is higher when clusters are dense and well separated, which relates to a standard concept of a cluster.    


Con:
The Silhouette Coefficient is generally higher for convex clusters than other concepts of clusters, such as density based clusters like those obtained through DBSCAN.


In [44]:
results.sort_values(by=['Sihouette_score'], ascending=False).head(3)

,Feature,Cluster,Name,Sihouette_score,DaviesBouldin_score,CalinskiHarabasz_score,InterClassDistance,IntraClassDistance,sse
8,fa,agg,fa_agg,0.509838,0.680138,666.729055,1376.069470,126.930530,1046.903383
6,fa,kmeans,fa_kmeans,0.480799,0.725760,762.691676,1390.848297,112.151703,1036.881215
3,pca,kmeans,pca_kmeans,0.388267,0.854907,372.052803,1289.797596,213.202404,1095.064840


## Davies-Bouldin Index

The minimum score is zero, with lower values indicating better clustering. 

Defintion:

- $s_i$: cluster diameter, the average distance between each point of cluster $i$ and the centroid of that cluster
- $d_{ij}$: distance between cluster centroids $i$ and $j$
- $R_{i,j}$: trades off $s_i$ and $d_{ij}$ as follows $R_{ij} = \dfrac{s_i+s_j}{d_{ij}}$
- $DB=\dfrac{1}{k}\sum^{k}_{i=1}max_{i\neq j}R_{ij}$



Pro:  
The computation of Davies-Bouldin is simpler than that of Silhouette scores.
The index is computed only quantities and features inherent to the dataset.


Con: 
- The Davies-Boulding index is generally higher for convex clusters than other concepts of clusters, such as density based clusters like those obtained from DBSCAN.
- The usage of centroid distance limits the distance metric to Euclidean space.    



In [45]:
results.sort_values(by=['DaviesBouldin_score'], ascending=True).head(3)

,Feature,Cluster,Name,Sihouette_score,DaviesBouldin_score,CalinskiHarabasz_score,InterClassDistance,IntraClassDistance,sse
8,fa,agg,fa_agg,0.509838,0.680138,666.729055,1376.069470,126.930530,1046.903383
6,fa,kmeans,fa_kmeans,0.480799,0.725760,762.691676,1390.848297,112.151703,1036.881215
4,pca,kmedoids,pca_kmedoids,0.386156,0.773924,303.459345,1249.726584,253.273416,1090.854126


## Calinski Harabasz (Variance Ratio Criterion)

Defintion: 

The index is the ratio of the sum of between-clusters dispersion and of inter-cluster dispersion for all clusters (where dispersion is defined as the sum of distances squared):

$s = \dfrac{tr(B_k)}{tr(W_k)} \times \dfrac{n_E - k}{k-1}$, where $n_E$ is number of data points, $k$ is number of clusters, $B_k$ (separation) is the between group (intercluster) dispersion matrix, $W_k$ (cohesion) is the intracluster dispersion matrix.


Pro:
- The score is higher when clusters are dense and well separated, which relates to a standard concept of a cluster.
- The score is fast to compute.


Con:
- The Calinski-Harabasz index is generally higher for convex clusters than other concepts of clusters, such as density based clusters like those obtained through DBSCAN.


In [46]:
results.sort_values(by=['CalinskiHarabasz_score'], ascending=False).head(3)

,Feature,Cluster,Name,Sihouette_score,DaviesBouldin_score,CalinskiHarabasz_score,InterClassDistance,IntraClassDistance,sse
2,feature,agg,feature_agg,0.256010,1.210290,3377.010997,522053.686651,9507.313349,1.696469e+06
0,feature,kmeans,feature_kmeans,0.247317,1.486733,3035.738793,521006.114283,10554.885717,1.696473e+06
1,feature,kmedoids,feature_kmedoids,0.216579,1.812765,2696.995064,519709.972268,11851.027732,1.696475e+06


# Visualizations

In [56]:
import torch
import torchvision
from torch.utils.tensorboard import SummaryWriter
from torchvision import datasets, transforms

import tensorflow as tf
import tensorboard as tb
tf.io.gfile = tb.compat.tensorflow_stub.io.gfile
# Writer will output to ./runs/ directory by default
def writer_name(name, metadata, feature):
    writer = SummaryWriter(comment=name)
    writer.add_embedding(np.array(feature),metadata = metadata)
    writer.close()

#%%

name_lst = ['feature_kmeans','feature_kmedoids','feature_agg', 
 'pca_kmeans', 'pca_kmedoids', 'pca_agg', 
 'fa_kmeans', 'fa_kmedoids', 'fa_agg',
 'lvf_kmeans', 'lvf_kmedoids', 'lvf_agg',
 'hcf_kmeans', 'hcf_kmedoids', 'hcf_agg']

feature_lst = [feature, feature, feature,
              feature_pca,feature_pca,feature_pca,
              feature_fa,feature_fa,feature_fa,
              feature_lvf,feature_lvf,feature_lvf,
              feature_hcf,feature_hcf,feature_hcf]


# meta_lst = [feature_kmeans, feature_kmedoids, feature_agg,
#             pca_kmeans, pca_kmedoids, pca_agg, 
#             fa_kmeans, fa_kmedoids, fa_agg,
#             lvf_kmeans, lvf_kmedoids, lvf_agg,
#             hcf_kmeans, hcf_kmedoids, hcf_agg]

#%%

for idx in range(len(name_lst)):
    name = name_lst[idx]
    feature =feature_lst[idx]
    metadata = df[name]
    writer_name(name, metadata, feature)
